### variable arguments ... 可变参数
https://docs.juliacn.com/latest/manual/functions/#变参函数<br>
varargs: 参数的个数可变

In [34]:
bar(a,b,x...) = (a,b,x)

bar(1,2) |> display
bar(1,2,3) |> display
bar(1,2,3,4,5,6) |> display

(1, 2, ())

(1, 2, (3,))

(1, 2, (3, 4, 5, 6))

**函数定义中都有 `x...`, 调用函数时:**<br>
+ 如果给的参数过多，**过多的参数都成 `x`(pack)**<br> 

+ **如果给`x`传递pack类数据(不限于元组), 函数体中会把 `x` unpack后使用** 

In [35]:
x = (3, 4)
bar(1,2,x...)

(1, 2, (3, 4))

In [36]:
x = (2, 3, 4)
bar(1,x...)

(1, 2, (3, 4))

In [37]:
x = (1, 2, 3, 4)
bar(x...)

(1, 2, (3, 4))

In [38]:
x = [1,2,3,4]
bar(x...)

(1, 2, (3, 4))

此外，**参数被放入的函数不一定是可变参数函数（尽管经常是），下面 baz(a,b)函数的参数就没有带...** 

In [40]:
baz(a,b) = a + b 
args = [1,2]
baz(args...)

3

In [41]:
args = [1,2,3]
baz(args...)

MethodError: MethodError: no method matching baz(::Int64, ::Int64, ::Int64)
The function `baz` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  baz(::Any, ::Any)
   @ Main e:\Projects.jl\Training.jl\7. function methods\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X35sZmlsZQ==.jl:1


正如你所见，****如果要拆解的容器（比如元组或数组）元素数量不匹配就会报错，和直接给多个参数报错一样。--- 函数定义时如果参数带...了，就没问题.**

#### 数量约束的可变参数
**Parametrically constrained Varargs methods 参数化约束的可变参数方法**
https://docs.juliacn.com/latest/manual/methods/#参数化约束的可变参数方法

### positional arguments 位置参数
即 basic form, 传值与函数参数之间按位置先后一一对应，不能混乱.

### optional arguments, 可选参数即默认参数
https://docs.juliacn.com/latest/manual/functions/#可选参数<br>
**不给值的 为默认值**

在很多情况下，函数参数有合理的默认值，因此也许不需要显式地传递。例如，Dates 模块中的 Date
```julia
function Date(y::Int64, m::Int64=1, d::Int64=1)
    err = validargs(Date, y, m, d)
    err === nothing || throw(err)
    return Date(UTD(totaldays(y, m, d)))
end
```


In [1]:
using Dates
Date(2000, 12, 12) |> display
Date(2000, 12) |> display
Date(2000) |> display

2000-12-12

2000-12-01

2000-01-01

#### 可选默认参数是使用多方法定义实现的
https://docs.juliacn.com/latest/manual/methods/#可选参数和关键字的参数的注意事项

In [7]:
ff(a=1,b=2) = a+2b

ff (generic function with 3 methods)

In [8]:
ff() |> display
ff(1,2) |> display

5

5

内部定义了
```julia
ff(a,b) = a+2b
ff(a) = ff(a,2)
ff() = ff(1,2)
```
这就意味着调用`f()`等于调用`f(1,2)`。在这个情况下结果是`5`，因为`f(1,2)`使用的是上面f的第一个方法。但是，不总是需要是这种情况。如果你定义了第四个对于整数更加专用的方法：

In [9]:
ff(a::Int,b::Int) = a-2b

ff (generic function with 4 methods)

In [10]:
ff() |> display
ff(1,2) |> display

-3

-3

此时`f()`和`f(1,2)`的结果都是`-3`。换句话说，可选参数只与函数捆绑，而不是函数的任意一个特定的方法。这个决定于使用的方法的可选参数的类型。当可选参数是用全局变量的形式定义时，可选参数的类型甚至会在运行时改变。

### keyword arguments 关键字参数
https://docs.juliacn.com/latest/manual/functions/#关键字参数<br>
参数的顺序可以打乱，不用严格按位置对应<br>

某些函数需要大量参数，按参数位置一一对应调用是麻烦的。关键字参数允许通过名称而不是仅通过位置来识别参数。


```julia
function plot(x, y; style="solid", width=1, color="black")
    ###
end
```
在函数调用时，分号是可选的：可以调用 `plot(x, y, width=2)` 或 `plot(x, y; width=2)`，但前者的风格更为常见。显式的分号只有在传递可变参数或下文中描述的需计算的关键字时是必要的。

```julia
function f(;x::Int=1)
    ###
end

function plot(x...; style="solid")
    ###
end

function f(x; y)
    ###
end
f(3, y=5) # ok, y is assigned
f(3)      # throws UndefKeywordError(:y)
```
在分号后也可传递 `key => value` 表达式。例如，`plot(x, y; :width => 2)` 等价于 `plot(x, y, width=2)`。当关键字名称需要在运行时被计算时，这就很实用了。

当计算可选和关键字参数的默认值表达式时，只有先前的参数才在作用域内。例如，给出以下定义：

```julia
function f(x, a=b, b=1)
    ###
end
```
`a=b` 中的 `b` 指的是外部作用域内的 `b`，而不是后续参数中的 `b`。

https://docs.juliacn.com/latest/manual/methods/#可选参数和关键字的参数的注意事项<br>

关键字参数与普通的位置参数的行为很不一样。特别地，他们不参与到方法分派中。**方法只基于位置参数分派，在匹配得方法确定之后关键字参数才会被处理。**

### Argument destructuring 参数解包
https://docs.juliacn.com/latest/manual/functions/#参数解构

In [22]:
(min, max) = (2, 10)
min |> display
max |> display

(min, max) = (22, 100, 111)
max

2

10

100

In [23]:
min2, max2 = (2, 10)
min2

2

In [24]:
(min3, max3) = 1,3,5
max3

3

The destructuring feature can also be used within a function argument. If a function argument name is written as a tuple (e.g. `(x, y)`) instead of just a symbol, then an assignment `(x, y) = argument` will be inserted for you:<br>

解构特性也可以在函数参数中使用。如果一个**函数的参数名**被写成一个元组（例如` (x, y)` ），而不是一个符号，那么一个赋值 **`(x, y) = argument(传值)`** 会为你插入：

In [21]:
minmax(x, y) = (y < x) ? (y, x) : (x, y)
gap((min, max)) = max - min

minmax(10, 2) |>display

gap(minmax(10, 2)) |> display
gap((2,10)) |> display
gap((20,100,200)) |> display

gap(2,10) |> display

(2, 10)

8

8

80

MethodError: MethodError: no method matching gap(::Int64, ::Int64)
The function `gap` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  gap(::Any)
   @ Main e:\Projects.jl\Training.jl\7. function methods\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W1sZmlsZQ==.jl:2


Notice the extra set of parentheses in the definition of gap. Without those, gap would be a two-argument function, and this example would not work.<br>

注意 gap 定义中的额外括号集。如果没有这些， gap 将是一个双参数函数，并且这个示例将无法工作。

Similarly, property destructuring can also be used for function arguments:<br>
类似地，属性解构也可以用于函数参数：
https://docs.julialang.org/en/v1/manual/functions/#Property-destructuring

In [25]:
(; b, a) = (a=1, b=2, c=3)

(a=1, b=2, c=3) |> typeof |> display
propertynames((a=1, b=2, c=3)) |> display

a |> display
c |> display

@NamedTuple{a::Int64, b::Int64, c::Int64}

(:a, :b, :c)

1

UndefVarError: UndefVarError: `c` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [28]:
(b, a) = (a=1, b=2, c=3)  # 去掉分号不对
a |> display
b |> display

2

1

https://docs.julialang.org/en/v1/manual/functions/#man-argument-destructuring

In [29]:
foo((; x, y)) = x + y
foo((x=1, y=2))

3

In [30]:
struct A
    x
    y
end

propertynames(A(3, 4)) |> display

foo(A(3, 4))

(:x, :y)

7

For anonymous functions, destructuring a single argument requires an extra comma:<br>
对于匿名函数，解构单个参数需要一个额外的逗号：

In [31]:
map(
    ((x, y),) -> x + y, 
    [(1, 2), (3, 4)]
    )

2-element Vector{Int64}:
 3
 7